In [76]:
# Run on first instance to install required libraries
%pip install smart_open
%pip install minecart
%pip install textract-trp

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [77]:
import time 
import re
import os
import trp
import boto3
import minecart
import json
import logging 

import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session

## Table Shaving
**Removing blank/empty rows that are reported in the line items**

In [78]:
def column_purge(df:pd.DataFrame) -> pd.DataFrame:
    """
    Column designed to filter out rows that are NaN (empty) and reduce dataframe size
    ------------------------------------------------------------------------------------
    Input
        :param data: (type pandas.DataFrame)
            A dataframe object that corresponds to the X-17A-5 filings
    
    Output
        :return: (type pandas.DataFrame)
            Returns a dataframe of size less than or equal to the original input 
    """
    # begin by filtering out the NaN rows present in the first column
    first_col = df.columns[0]
    new_df = df[np.isin(df[first_col], df[first_col].dropna())]    # select subset of rows 
    
    # replace any missing NaN values with an empty string
    new_df = new_df.fillna('')
    
    return new_df

## Table column merging
**For tables with three columns we merge the last two columns into a once unique column**

In [79]:
def merge(df:pd.DataFrame) -> pd.DataFrame:
    """
    Function passes a special dataframe, and reduces its dimensions accordingly. 
    Example releases include, but are note limited to, 1224385-2016 and 72267-2003
    ------------------------------------------------------------------------------------
    Input
        :param data: (type pandas.DataFrame)
            A dataframe object that corresponds to the X-17A-5 filings
    
    Output
        :return: (type pandas.DataFrame)
            Returns a dataframe of of size (Nx3) -> (Nx2)
    
    e.g.
    
    Converts a wide dataframe, balance sheet into a smaller rectangular form
                  0                                                 1                 2
            ====================================================================================
        0   Assets                                          | NaN            | NaN  
        1   Cash and cash equivalents                       | $ 606,278      |     
        2   Cash and securities segregated pursuant         | 273,083        | 
        3   Collateralized short-term financing agreements: | NaN            | $ 1,345
    
    
    Rectangular form of the the dataframe ->
                   0                                                 1          
            =====================================================================
        0   Assets                      
        1   Cash and cash equivalents                       | $ 606,278        
        2   Cash and securities segregated pursuant         | 273,083        
        3   Collateralized short-term financing agreements: | $ 1,345            
    """
    # work on itterative merging for rows, check left/right and top/bottom
    n = df.shape[0]
    trans = []

    for i in range(n):
        row = df.iloc[i]         # index into the row

        name = row.iloc[0]       # the line item name (e.g. Total Assets)
        col1 = row.iloc[1]       # the first value(s) column
        col2 = row.iloc[2]       # the second value(s) column 
        
        # ----------------------------------------------
        # NOTE: We say nothing if both col 1 and 2 are 
        #       both populated with a value
        # ----------------------------------------------
        
        if col1 is not np.nan:
            trans.append([name, col1])  # if column 1 has a value we take it by default
        elif col2 is not np.nan:
            trans.append([name, col2])  # if column 1 has no value, but column 2 does, we take it
            
        # ----------------------------------------------
        
        # we want to check if there exists two NaNs - is it real or false flag
        if (col1 is np.nan) and (col2 is np.nan): 
            
            # look up one row (if possible to see if col1 and col2 are populated)
            try:
                # check the information for the above row
                prior_row = df.iloc[i-1]                     # previous dataframe row 
                prior_col1 = prior_row.iloc[1]               # first column from previous row
                prior_col2 = prior_row.iloc[2]               # second column from previous row
                
                # if both values present then we simply use the right hand side value above  
                if (prior_col1 is not np.nan) and (prior_col2 is not np.nan):
                    trans.append([name, prior_col2])
            
            # IndexError if not possible to look up one row       
            except IndexError: pass
    
    return pd.DataFrame(trans)

In [80]:
def column_merge(df:pd.DataFrame) -> pd.DataFrame:
    """
    Function determines whether a Balance Sheet should be merged or simply filtered. 
    Our two cases are determined as follows:
        * If the second column present in the balance sheet is mostly empty we assume that 
          the second column is an aggregated column, and we can merge it
        * However, if the second column is mostly filled with values, we assume that this
          represents accounting figures from the previous year 
    ------------------------------------------------------------------------------------
    Input
        :param data: (type pandas.DataFrame)
            A dataframe object that corresponds to the X-17A-5 filings
    
    Output
        :return: (type pandas.DataFrame)
            Returns a dataframe of of size (Nx3) -> (Nx2)
    """
    # two events could occur at this point (either the column represents totals, or values from a prior-year)
    arr = df[df.columns[2]].values

    # check the scope of the second column 
    n = arr.size
    k = arr.tolist().count(np.nan)

    # k-check: if more than half the arr size is np.nan we assume this is a "fake column"
    # we merge these columns since there are many blank rows, otherwise we assume year split 
    if k/n >= 0.50:
        new_df = merge(df)            # merge rows by merge function
    else:
        new_df = df[df.columns[:2]]   # return the most recent year 
    
    return new_df

## Table Row Split
**Since many of the existing tables run the risk of overlapping rows we work to split these rows to appropriate values**

In [81]:
def row_split(df:pd.DataFrame, text_file:dict) -> pd.DataFrame:
    """
    Function designed to split conjoined rows from Balance sheet dataframes into individual rows.
    Example releases include, but are note limited to, 42352-2015, 58056-2009, 58056-2013, 58056-2019
    ------------------------------------------------------------------------------------
    Input:
        :param df: (type pandas dataframe)
            References the balance sheet dataframe read in from AWS Textract
        :param text_file: (type dictionary)
            Stores text values with corresponding confidence level for balance sheet pages
    
    Output:
        :param return: (type pandas dataframe) 
            A processed dataframe of size greater than or equal to the inputed dataframe
    """
    
    # ##############################################################
    # NESTED HELPER FUNCTIONS
    # ##############################################################
    
    def find_row_splits(val) -> bool:
        """
        Compute a boolean measure to assess whether a row is conjoined or not. We make
        the assumption that a row is conjoined or merged if there exists a space in the 
        first value column (omiting the dollar sign $)
        """
        try:
            # split the data figures for each balance sheet figure
            arr = val.split(' ')
            
            # remove the $ sign if present in the list (this helps avoid false pasitives) 
            arr = list(filter(lambda x: x != '$', arr))
            
            # if length of read list exceeds 1 then we know there exists a multi-row bunch
            if len(arr) > 1:
                return True
            else: return False
        
        # handle exception for NaN (no attribute to split) 
        except AttributeError: return False
    
    def extract_lineitems(line:list, value:list, dictionary:dict) -> list:
        """
        Extract the appropriate line items from each line value.
        """
        splits = []
        
        # iterate through each line item
        for i in dictionary.keys():
    
            # we check for real key-value names avoiding single character keys
            if len(i) > 1: 
                idx = line.find(i)    # find the index of key-value (if possible) in line item array

                # if we find such a value we append the series (failure to find results idx = -1)
                if idx >= 0: splits.append(i)
        
        # check whether we have a one-to-one mapping between line items and line values, 
        # e.g. ['Assets', 'Cash', 'Recievables'] -> ['1,233', '4,819'] (3x2 mapping)
        n = len(splits) - len(value)
        
        # there exists a mismatch (more line items terms)
        if n > 0: 
            return splits[n:]
        # we assume there doesn't exist a negative mismatch (more values than items)
        else: 
            return splits
    
    def recursive_splits(values:list, lineName:list, sub=[]) -> pd.DataFrame:
        """
        Recursively breaks up merged rows for each split until no merged row is left
        """
        # if our list exceeds 1 in length, we continue to split
        if len(values) > 1:
            # construct a dataframe row of the first split term to append to sub list
            row = pd.DataFrame([lineName[0], values[0]]).T
            sub.append(row)
            
            # we pass the +1 index splits and line name, appending the first-most layer 
            return recursive_splits(values[1:], lineName[1:], sub=sub)
        else:
            row = pd.DataFrame([lineName[0], values[0]]).T
            sub.append(row)
            
            # we concatenate all DataFrames vertically to form a large DataFrame 
            return pd.concat(sub)
        
    # ##############################################################
    # ##############################################################    
    
    # select all the rows that match our description, where a space exists = row merge 
    selections = df[df[df.columns[1]].apply(lambda x: find_row_splits(x))]
    idxs = selections.index
    
    # iterate through each row that is determined to be conjoined
    for i in idxs:
        
        # slice dataframe according to the idx selection (we search for all periods were a break occurs)
        top = df.loc[:i-1]
        bottom = df.loc[i+1:]
        
        # divide the identified term from the selection e.g. "$ 9,112,943 13,151,663" -> ["$", "9,112,943", "13,151,663"] 
        # and filter out the $ sign in the list e.g. ["$", "9,112,943", "13,151,663"] -> [9,112,943", "13,151,663"]
        values = df[df.columns[1]].loc[i].split(' ')
        values = list(filter(lambda x: x != '$', values))
        
        # extract line names according to Text parsed list (requires parsed TEXT JSON)
        # e.g. ['Securities Held Total Assets'] -> ['Securities Held', 'Total Assets']
        lineName = df[df.columns[0]].loc[i]
        lineName = extract_lineitems(lineName, values, text_file)
        
        # determine the splits for the corresponding row
        mid = recursive_splits(values, lineName, sub=[])
        mid.columns = df.columns

        # reassign the value of df2 to update across each iteration
        df = pd.concat([top, mid, bottom])
        
    return df

## Numeric Conversion
**Work on converting all string and poor formating quantities to numerical type**

In [82]:
def num_scale(text_dict:dict) -> float:
    """
    Function used for scaling accounting figures by reported unites
    """
    scalar = {'thousands': 1e3, 'hundreds':1e2, 'millions':1e6, 'billions': 1e9}
    
    # iterate through each of the text values from dictionary map
    for text_value in text_dict.keys():
        
        # we check to see whether the text is found in our scalar dictionary
        for scale_type in scalar.keys():
            
            # search for the presence of the scale identifier (e.g. millions) 
            scale_search = re.search(scale_type, text_value, flags=re.I)
            if scale_search is not None:
                return scalar[scale_type]              # if found we simply return the multiplier
    
    # if failure reached then assume no multiplier (1)
    return 1

In [83]:
def num_strip(number):
    """
    This function converts a string to a numeric quantity, handles weird string format. 
    We handle input arguments of a string, integer or numpy.ndarray
    """
    
    numType = type(number)

    # if provided a non-empty string, perform regex operation 
    if (numType is str) and (len(number) > 0):

        # check for accounting formats that use parenthesis to signal losses 
        if number[0] == '(': number = '-' + number

        # case replacing to handle poor textract reading of numbers
        number = number.replace('I', '1').replace('l', '1')

        # --------------------------------------------------------------
        # Explanation of the Regex Expression:
        #      [^0-9|.|-]     = match all elements that are not numeric 0-9, periods "." or hyphens "-"
        #      (?<!^)-        = match all elements that are hyphens "-" not in the first index position
        #      \.(?=[^.]*\.)  = match all elements that are periods "." except the last instance
        # --------------------------------------------------------------

        check1 = re.sub("[^0-9|.|-]", "", number)         # remove all the non-numeric, periods "." or hyphens "-"
        check2 = re.sub("(?<!^)-", "", check1)            # removes all "-" that aren't in the first index 
        check3 = re.sub("\.(?=[^.]*\.)", "", check2)      # removes all periods except the last instance of "." 

        # --------------------------------------------------------------

        # we consider weird decimal values that exceed 2 spaces to the right (e.g. 432.2884)
        period_check = check3.find('.')                         # returns the location of the period 
        right_tail_length = len(check3) - period_check - 1      # right-tail length should not exceed 2

        # if more than 2 trailing digits to decimal point we assume incorrect placement
        if right_tail_length > 2:
            check3 = check3.replace('.', '')

        # last check against poor lagging formats e.g. "." or "-" to return nan or floating-point number
        if (check3 == '-') or (check3 == '.'):
            return 0.0
        else:
            # try to cast to floating point value, else flat NaN
            try: 
                return float(check3)
            except ValueError: 
                return np.nan

    # if operator is an integer or float then simply return the value
    elif (numType is int) or (numType is float):
        return number

    else:
        return np.nan

In [84]:
def cleanNumeric(value):
    """
    This function is a wrapper for calling the numerical extraction function 
    ------------------------------------------------------------------------------------
    Input:
        :param value:
            String with hidden numeric quanity (e.g. $ 19,225 = 19255)  
        :param text_file: (type dictionary)
            Stores text values with corresponding confidence level for balance sheet pages
            
    Output:
        :param return:
            A processed numeric quantiity or numpy.nan value depending on string issues  
    """
    
    assert type(value) is str or int or np.ndarray, 'Value must be of type string, integer, float or numpy array'
    
    # checks to see what type of value is being provided
    operator = type(value)
    
    # if provided a string, perform regex operation 
    if (operator is str) and (len(value) > 0):
        return num_strip(value)
    
    # if operator is integer then simply return the value, no need to modify 
    elif (operator is int) or (operator is float):
        return value 
    
    # if operator is numpy array then we perform a extraction per element in array
    elif (operator is np.ndarray):
        vFunc = np.vectorize(num_strip)      # vectorize function to apply to numpy array
        cleanValue = vFunc(value)            # apply vector function
        return cleanValue 


### Final Main Execution

In [85]:
if __name__ == "__main__":
    
    # initiate s3 bucket and corresponding data folder
    bucket = 'ran-s3-systemic-risk'
    data_folder = 'Output/X-17A-5-BS/'
    temp_folder = 'Temp/'

    # Amazon Textract client and Sagemaker session
    textract = boto3.client('textract')
    s3 = boto3.client('s3')
    session = Session()
    
    # retrieving text JSON file from s3 bucket and store to temp 
    s3.download_file(bucket, 'Temp/X17A5-TEXT.json', 'temp2.json')

    # read data on TEXT-Confidence dictionary
    with open('temp2.json', 'r') as f: text_dictionary = json.loads(f.read())  

    # remove local files for JSON
    os.remove('temp2.json')
    
    # csv directory with all X-17A-5 balance sheet information 
    paths = np.array(session.list_s3_files(bucket, data_folder))[1:]
    
    # iterate through each csv path
    for csv in paths:
        
        fileName = csv.split('/')[-1]         # strip filename from each csv
        base_file = fileName.split('.')[0]    # CIK-YYYY-MM-DD base name

        # download X-17A-5 csv file as a temporary csv file  
        s3.download_file(bucket, csv, 'temp.csv')
        df = pd.read_csv('temp.csv')

        # re-assign dataframe of balance sheet after cleanse
        df = column_purge(df)
        
        # --------------------------------------------------------------------------------------------------
        # COLUMN MERGING (IF NECESSARY)
        # --------------------------------------------------------------------------------------------------
        
        # if columns greater than 2, we have a weird data table that needs to be "merged"
        # NOTE: By construction we never have more than 3 columns present, thanks to our Textract check 
        if df.columns.size > 2:
            df = merge(df)
            print('We merged the columns of {}'.format(fileName))
        
        # --------------------------------------------------------------------------------------------------
        # ROW SPLIT FOR MERGED ROWS (IF NECESSARY)
        # --------------------------------------------------------------------------------------------------
        
        # check for presence of row splits and correct any if found 
        tempDF = row_split(df, text_dictionary[base_file])
        
        # if difference is found in shape, then a transformation was done 
        if tempDF.shape != df.shape:
            print("Fixed the merged rows for {}".format(fileName))
        
        # --------------------------------------------------------------------------------------------------
        # NUMERIC CONVERSION
        # --------------------------------------------------------------------------------------------------
        
        # pass numeric converter to the column to convert string to numerics
        tempDF[tempDF.columns[1]] = tempDF[tempDF.columns[1]].apply(cleanNumeric)
        
        # remove any NaN rows post numeric-conversion
        postDF = tempDF.dropna().copy()
        
        # check for potential scaler multipler on cash flows (adjust multiplier if possible)
        scale = num_scale(text_dictionary[base_file])
        postDF[postDF.columns[1]] = postDF[postDF.columns[1]].apply(lambda x: x * scale)
        
        print('We converted to numeric figures for {}\n'.format(fileName))
        
        # --------------------------------------------------------------------------------------------------
        # BALANCE SHEET STORAGE
        # --------------------------------------------------------------------------------------------------

        print(postDF)
#         # writing data frame to .csv file
#         postDF.to_csv(fileName, index=False)

#         # save contents to AWS S3 bucket
#         with open(fileName, 'rb') as data:
#             s3.put_object(Bucket=bucket, Key=data_folder + fileName, Body=data)

#         # remove local file after it has been used
#         os.remove(fileName)
#         os.remove('temp.pdf')

    print('\nAll .csv files are cleaned and primed')

We converted to numeric figures for 1224385-2004-03-01.csv

                                                    0             1
0                                                Cash  1.760000e+05
1     Cash segregated pursuant to federal regulations  7.500000e+07
2     Securities purchased under agreements to resell  7.944113e+09
3   Securities owned, at market value ($8,769,300 ...  9.075170e+09
4   Receivable from broker-dealers and clearing or...  4.049708e+09
5                           Receivable from customers  5.124600e+07
6                         Accrued interest receivable  4.445700e+07
7   Property, equipment, and leasehold improvement...  1.106700e+07
8                                       Goodwill, net  6.112000e+06
9                                        Other assets  2.527380e+08
10                                       Total assets  2.150979e+10
13     Securities sold under agreements to repurchase  1.360247e+10
14  Securities sold, not yet purchased, at market ...  3

We converted to numeric figures for 1224385-2010-03-12.csv

                                                    0             1
0                                                Cash  3.083100e+07
1     Cash segregated pursuant to federal regulations  2.500000e+07
2   Financial instruments owned, at fair value ($7...  9.048848e+09
3   Receivable from broker-dealers and clearing or...  4.674419e+09
4     Securities purchased under agreements to resell  3.465845e+09
5                                            Goodwill  7.968700e+07
6                           Receivable from customers  5.034900e+07
7   Property, equipment, and leasehold improvement...  1.119000e+06
8                                        Other assets  1.235600e+08
9                                        Total assets  1.749966e+10
12     Securities sold under agreements to repurchase  7.675431e+09
13  Financial instruments sold, not yet purchased,...  4.339193e+09
14  Payable to broker-dealers and clearing organiz...  9

We converted to numeric figures for 1224385-2018-02-28.csv

                                                    0             1
0                                                Cash  2.466460e+08
1     Cash segregated pursuant to federal regulations  1.493144e+09
2                                 Securities borrowed  3.852755e+10
3     Securities purchased under agreements to resell  1.434480e+10
4   Receivable from broker-dealers and clearing or...  2.489979e+10
5                      Receivable from customers, net  4.179195e+09
6   Financial instruments owned, at fair value ($4...  4.715217e+10
7   Property, equipment, and leasehold improvement...  7.400000e+05
8                                            Goodwill  7.968700e+07
9                                        Other assets  4.784850e+08
10                                       Total assets  1.314022e+11
13     Securities sold under agreements to repurchase  7.075811e+10
14                                  Securities loaned  1

We converted to numeric figures for 42352-2003-01-28.csv

                                                    0             1
0                           Cash and cash equivalents  2.816419e+09
2                               and other regulations  1.656822e+10
3   Receivables from brokers, dealers and clearing...  3.964833e+09
4       Receivables from customers and counterparties  1.026552e+10
5                                 Securities borrowed  1.057280e+11
6     Securities purchased under agreements to resell  3.239469e+10
7          Financial instruments owned, at fair value  3.204794e+10
8   Financial instruments owned and pledged as col...  8.480307e+09
9    Total financial instruments owned, at fair value  4.052825e+10
10                                       Other assets  1.716894e+09
13                              Short-term borrowings  3.435200e+10
14  Payables to brokers, dealers and clearing orga...  3.122873e+09
15           Payables to customers and counterparties  5.1

We converted to numeric figures for 42352-2010-03-01.csv

                                                    0             1
0                           Cash and cash equivalents  3.273000e+09
1   Cash and securities segregated for regulatory ...  1.145900e+10
3   Securities borrowed (includes $79,137 at fair ...  2.124910e+11
4   Financial instruments purchased under agreemen...  7.049900e+10
5   Receivables from brokers, dealers and clearing...  8.193000e+09
6   Receivables from customers and counterparties ...  2.020300e+10
7          Financial instruments owned, at fair value  1.131040e+11
8   Financial instruments owned and pledged as col...  1.963900e+10
9    Total financial instruments owned, at fair value  1.327430e+11
10                                       Other assets  4.894000e+09
11                                       Total assets  4.637550e+11
14   long-term borrowings (includes $3 at fair value)  1.527600e+10
16  Securities loaned (includes $23,810 at fair va...  6.6

IndexError: list index out of range